# **First Python Data Analysis Project**

### ***In this project I will be trying to do some basic analysis with the Formula One data to test by skills in Python data analysis which I have been learning over the past month. Being an Formula One fan I thought working with the huge amount of data from this sport would be enjoyable. I will be using the F1 dataset "Formula 1 World Championship (1950 - 2022)" which is available in Kaggle.***

## **Importing Libraries**

In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline
import numpy as np
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['figure.facecolor'] = '#00000000'


## **Loading the dataset**

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data_dir ='../input/formula-1-world-championship-1950-2020'

In [ ]:
drivers_raw_df = pd.read_csv(f'{data_dir}/drivers.csv')
qualifying_raw_df = pd.read_csv(f'{data_dir}/qualifying.csv')
laptimes_raw_df = pd.read_csv(f'{data_dir}/lap_times.csv')
pitstops_raw_df = pd.read_csv(f'{data_dir}/pit_stops.csv')
constructor_standings_raw_df = pd.read_csv(f'{data_dir}/constructor_standings.csv')
seasons_raw_df = pd.read_csv(f'{data_dir}/seasons.csv')
circuits_raw_df = pd.read_csv(f'{data_dir}/circuits.csv')
constructors_raw_df = pd.read_csv(f'{data_dir}/constructors.csv')
driver_standings_raw_df = pd.read_csv(f'{data_dir}/driver_standings.csv')
results_raw_df = pd.read_csv(f'{data_dir}/results.csv')
status_raw_df = pd.read_csv(f'{data_dir}/status.csv')
races_raw_df = pd.read_csv(f'{data_dir}/races.csv')

#### Getting some info about the data

In [ ]:
results_raw_df.info()

#### Results data connects every other data frame as it has the key from other data frames.

#### Making a copy of the data frames by dropping the columns that we are not going to use for analysis in this project. We will also rename the columns to be more specific.

In [ ]:
races_df = races_raw_df.copy()
races_df = races_df.drop(columns = ['url',
       'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'time'])
races_df = races_df.rename(columns ={'name':'race_name'})

In [ ]:
circuits_df = circuits_raw_df.copy()
circuits_df =circuits_df.drop(columns = ['lat', 'lng','alt', 'url'])
circuits_df = circuits_df.rename(columns={'name':'circuit_name', 'location':'city'})

In [ ]:
drivers_df = drivers_raw_df.copy()
drivers_df =drivers_df.drop(columns =['driverRef', 'number', 'code', 'url'])
drivers_df['driver_name'] = drivers_df['forename'] + ' ' + drivers_df['surname']
drivers_df = drivers_df.drop(columns =['forename', 'surname'])

In [ ]:
constructors_df=constructors_raw_df.copy()
constructors_df =constructors_df.drop(columns = ['url','constructorRef'])
constructors_df = constructors_df.rename(columns = {'name':'constructors_name'})

In [ ]:
results_copy_df = results_raw_df.copy()

#### Merging the data frames that we plan to use for getting a better picture

In [ ]:
merged_df = results_copy_df.merge(status_raw_df , on = 'statusId')
merged_df = merged_df.merge(races_df, on = 'raceId')
merged_df = merged_df.merge(drivers_df, on = 'driverId')
merged_df = merged_df.merge(constructors_df, on = 'constructorId')
merged_df = merged_df.merge(circuits_df , on = 'circuitId')

In [ ]:
merged_df.info()

In [ ]:
merged_df = merged_df.rename(columns= {'nationality_x':'driver_nationality','nationality_y':'constructor_nationality'})

In [ ]:
merged_df.describe()

#### Describe() function on a very big data does not give any picture except returining a few numbers without much context so we have to dig deep and be specific in our analysis 

## **Driver Analysis**

#### Stripping the driver data from merged data frame year wise to get some insights.

In [ ]:
driver_analysis_df = merged_df.groupby(['year','driver_name']).agg({'points': ['sum'],'raceId':['count'], 'positionOrder':['mean','std'] }).reset_index()


In [ ]:
driver_analysis_df.info()

#### The resultant data frame has multilevel index and we will sort that out and rename the columns to perform more analysis

In [ ]:
driver_analysis_df.columns = ['_'.join(col).strip() for col in driver_analysis_df.columns.values]
driver_analysis_df = driver_analysis_df.rename( columns = {'year_':'year', 'driver_name_' : 'driver_name'})

#### With the above data frame lets get the driver with most points in each year which will get us the driver champion of that year.

In [ ]:
champion_df= driver_analysis_df.groupby(['year', 'driver_name']).agg({'points_sum':sum}).reset_index()

In [ ]:
champion_df = champion_df.sort_values(['year','points_sum'], ascending = False).groupby('year').head(1)

#### We will drop the 2022 year as it is the current year and the season is on going and the results are not final

In [ ]:
champion_df = champion_df.drop(3155)

#### Champions of the last 20 Years 

In [ ]:
champion_df[['year','driver_name']].head(20)

#### Top 10 drivers with most number of race entries in Formula 1 history

In [ ]:
most_races = merged_df.groupby('driver_name')[['raceId']].count().reset_index()
most_races = most_races.sort_values('raceId', ascending= False).head(10)
most_races = most_races.rename(columns ={'raceId': 'total_races'})
most_races

#### Top 10 drivers who has done most number of laps in Formula 1 history

In [ ]:
most_laps = merged_df.groupby('driver_name')[['laps']].sum().reset_index()
most_laps = most_laps.sort_values('laps', ascending = False).head(10)
most_laps

#### To compare drivers across different generations we will plot a graph using the average and standard deviation of positions finished by the drivers to see who was able to produce a more consistent performance.

In [ ]:
# Retreiving the relavent data of position finished and narowwing down to drivers who has taken part in more than 100 races.

driver_analysis_df2 = merged_df.groupby('driver_name').agg({'raceId':['count'], 'positionOrder':['mean', 'std']}).reset_index()
driver_analysis_df2.columns = ['_'.join(col).strip() for col in driver_analysis_df2.columns.values]
driver_analysis_df2 = driver_analysis_df2[driver_analysis_df2.raceId_count > 100]
driver_analysis_df2 = driver_analysis_df2.sort_values('positionOrder_mean').head(10)
driver_analysis_df2 = driver_analysis_df2.rename(columns = {'driver_name_':'driver_name'})



In [ ]:
# Plotting a scatter plot with the above data 

plt.figure(figsize = (12,8))
plt.title('Average position finished vs Standard Deviation of position finished')
sns.scatterplot(x = 'positionOrder_mean' , y = 'positionOrder_std' , hue = 'driver_name' ,s=100 , data = driver_analysis_df2);

#### From the graph we can see that Lewis Hamilton clearly stands out even among some of the top ten drivers of the world over the years. This graph shows his consistency of finishing in the top. The closest to him comes Max Verstappen closely followed by Michel Schumacher the fellow seven time world champion with Lewis . 

## **Constructor Analysis** 

#### Now we will look into some data from constructors by extracting them from the merged data frame and performing a similar analysis to that of driver analysis.

In [ ]:
con_analysis_df = merged_df.groupby(['year','constructors_name']).agg({'points': ['sum'],'raceId':['count'],'positionOrder':['mean','std'] }).reset_index()
con_analysis_df.columns = ['_'.join(col).strip() for col in con_analysis_df.columns.values]
con_analysis_df.info()

In [ ]:
con_analysis_df = con_analysis_df.rename(columns ={'year_':'year','constructors_name_':'constructors_name'})

#### Getting Constructors champions year wise. 

In [ ]:
con_champion_df= con_analysis_df.groupby(['year', 'constructors_name']).agg({'points_sum':sum}).reset_index()
con_champion_df = con_champion_df.sort_values(['year','points_sum'], ascending = False).groupby('year').head(1)

# Dropping the data of current year as the results are not final
con_champion_df = con_champion_df.drop(1089)

#### List of Constructors champions in the last 20 years 

In [ ]:
con_champion_df[['year','constructors_name']].head(20)

#### In order to get the number of race entries by the constructors we have to group by constructors name and race ID and again group by constructors name as for every race the constructor enters two cars in its team name. We will also get average and standard deviation of positions finished to plot a graph.

In [ ]:
con_analysis_df2 = merged_df.groupby(['constructors_name', 'raceId']).agg({'positionOrder':['mean', 'std']}).reset_index()

con_analysis_df2.columns = ['_'.join(col).strip() for col in con_analysis_df2.columns.values]

con_analysis_df3 = con_analysis_df2.groupby(['constructors_name_']).agg({'constructors_name_':['count'],'positionOrder_mean':['mean'] , 'positionOrder_std' :['mean']}).reset_index()

con_analysis_df3.columns = ['_'.join(col).strip() for col in con_analysis_df3.columns.values]

con_analysis_df3 = con_analysis_df3.rename(columns = {'constructors_name__' : 'constructors_name',
                                                      'constructors_name__count' : 'constructor_entry',
                                                      'positionOrder_mean_mean' : 'positionOrder_mean', 
                                                      'positionOrder_std_mean' : 'positionOrder_std'})

#### Top 20 Constructors with most number of race entries over the years.

In [ ]:
x = con_analysis_df3.sort_values('constructor_entry', ascending = False).head(20)
x[['constructors_name','constructor_entry']]

#### For getting the top performing constructor we narrow down the constructors with at least 200 race entries and sort by their average position finishes. 

In [ ]:
con_analysis_df3 = con_analysis_df3[con_analysis_df3.constructor_entry > 200]
con_analysis_df3 = con_analysis_df3.sort_values('positionOrder_mean').head(10)

In [ ]:
# Plotting a graph with the above data

plt.figure(figsize = (12,8))
plt.title("Constructor's Average position finished vs Standard Deviation of position finished")
sns.scatterplot(x = 'positionOrder_mean' , y = 'positionOrder_std' , hue = 'constructors_name' ,s=100 , data = con_analysis_df3);

#### From the above graph we can see that in the short period Mercedes has yielded impressive result. If we recall from driver analysis graph Lewis Hamiliton who has been the longest serving driver for Mercedes for 8 years showed similar result of consistency, implying this as one of the best partnership of right driver and right team. Driver analysis graph also had Nico Rosberg and Valtteri Bottas with impressive results who were drivers of Mercedes showing the dominance of Mercedes in the top.

#### From the graph we can see Ferrari comes in top 3 closely following Redbull. Ferrari with most number of entries in Formula 1 with almost four times the number of entries as Mercedes and featuring in top 3 is monumental, it shows Ferrari's consistency and all round performance over 70 years in Formula 1. Driver analysis also had a lot of Ferrari driver like that of Michael Schumacher, Fernando Alonso , Alain Prost who had a short stint with the team and Sebastian Vettel who also drove for Redbull which also features in top 3 constructors according to the graph.

####  Both driver analysis and constructor analysis charts are mostly topped by Mercedes, Ferrari and Redbull team and drivers.

## **Circuit Analysis** 

#### In Circuit analysis we look how the laptimes has changed in the circuits over the years getting an brief insight about how the speed of cars has changed over the years. 

In [ ]:
# Merging the data we require from different data frame

fastestlap_df = races_df.merge(circuits_df , on = 'circuitId')
fastestlap_df = fastestlap_df.merge(results_copy_df, on = 'raceId')

In [ ]:
fastestlap_df['fastestLapTime']

#### The fastest lap data is not in a format that can be used for analysis. In order to make it usable we are going to strip the data separately and convert it to numeric and add them together. 

In [ ]:
fastestlap_df['f_lap_1'] = fastestlap_df['fastestLapTime'].apply(lambda x : (x.split('.')[-1]))
fastestlap_df['f_lap_2'] = fastestlap_df['fastestLapTime'].apply(lambda x : (x.split('.')[0]))
fastestlap_df['f_lap_3'] = fastestlap_df['f_lap_2'].apply(lambda x: (x.split(':')[-1]))
fastestlap_df['f_lap_4'] = fastestlap_df['f_lap_2'].apply(lambda x: (x.split(':')[0]))

In [ ]:
# Removing empty spaces after stripping the data separately 

fastestlap_df['f_lap_1'] = fastestlap_df['f_lap_1'].str.strip()
fastestlap_df['f_lap_3'] = fastestlap_df['f_lap_3'].str.strip()
fastestlap_df['f_lap_4'] = fastestlap_df['f_lap_4'].str.strip()

In [ ]:
# Converting them to a numeric value

fastestlap_df['f_lap_1'] = pd.to_numeric(fastestlap_df['f_lap_1'] , errors = 'coerce')
fastestlap_df['f_lap_3'] = pd.to_numeric(fastestlap_df['f_lap_3'] , errors = 'coerce')
fastestlap_df['f_lap_4'] = pd.to_numeric(fastestlap_df['f_lap_4'] , errors = 'coerce')

#### Merging the data together again and we are merging in a way that we get the fastest lap in milliseconds as it will be easy to plot in a graph



In [ ]:
fastestlap_df['fastest_lap'] = fastestlap_df['f_lap_1'] + fastestlap_df['f_lap_3']*1000 + fastestlap_df['f_lap_4']*60*1000



In [ ]:
fastestlap_df['fastest_lap']

In [ ]:
fastestlap_df = fastestlap_df.drop(columns = ['f_lap_4','f_lap_3', 'f_lap_2','f_lap_1'])

In [ ]:
x = fastestlap_df.sort_values('year')

x.info()

#### It seems like the fastest lap column has some null values in it.

In [ ]:
x.groupby('year')[['fastest_lap']].max().head(10)

In [ ]:
x.groupby('year')[['fastest_lap']].max().tail(10)

In [ ]:
# Getting the first valid row in fastest lap data

print(x.fastest_lap.first_valid_index())

In [ ]:
x.loc[2516]

#### Fastest lap data is available only from the year 2004, so we will do analysis on fastest lap from year 2004 onwards.

In [ ]:
fastestlap_df = fastestlap_df[(fastestlap_df['year'].between(2004,2021, inclusive = 'both'))]

#### Circuits which has hosted the most number of races from 2004

In [ ]:
h = fastestlap_df.groupby(['year','circuit_name']).count().reset_index()
h['circuit_name'].value_counts()

#### Striping the data separately for 6 of the popular circuits that has hosted most number of races since 2004 

In [ ]:
lap_time_monza = fastestlap_df[fastestlap_df['circuit_name'] == 'Autodromo Nazionale di Monza']
lap_time_monaco = fastestlap_df[fastestlap_df['circuit_name']== 'Circuit de Monaco']
lap_time_silverstone = fastestlap_df[fastestlap_df['circuit_name']== 'Silverstone Circuit']
lap_time_catalunya = fastestlap_df[fastestlap_df['circuit_name']=='Circuit de Barcelona-Catalunya']
lap_time_hungaroring = fastestlap_df[fastestlap_df['circuit_name']== 'Hungaroring']
lap_time_spa = fastestlap_df[fastestlap_df['circuit_name']== 'Circuit de Spa-Francorchamps']

#### Getting the fastest lap and grouping them by year

In [ ]:
lap = lap_time_silverstone.groupby('year')[['fastest_lap']].min().reset_index()
lap1 =  lap_time_monaco.groupby('year')[['fastest_lap']].min().reset_index()
lap2 =  lap_time_monza.groupby('year')[['fastest_lap']].min().reset_index()
lap3 = lap_time_catalunya.groupby('year')[['fastest_lap']].min().reset_index()
lap4 = lap_time_hungaroring.groupby('year')[['fastest_lap']].min().reset_index()
lap5 = lap_time_spa.groupby('year')[['fastest_lap']].min().reset_index()

In [ ]:
# Plotting a line graph for Lap time over the years

plt.figure(figsize = (20,8))
plt.plot(lap.year,lap.fastest_lap)
plt.plot(lap1.year,lap1.fastest_lap)
plt.plot(lap2.year,lap2.fastest_lap)
plt.plot(lap3.year,lap3.fastest_lap)
plt.plot(lap4.year,lap4.fastest_lap)
plt.plot(lap5.year,lap5.fastest_lap)
plt.xticks(np.arange(min(lap.year),max(lap.year)+1 , 1.0))
plt.ylabel('Fastest lap in milliseconds')
plt.xlabel('Year')
plt.title('Lap time over the years')
plt.legend(['Silverstone', 'Monaco','Monza','Catalunya','Hungaroring','Spa']);

#### Plotting  a separate graph for each circuits.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 8))


axes[0,0].plot(lap.year,lap.fastest_lap)
axes[0,0].set_title('Silverstone')
axes[0,0].set_ylabel('Fastest lap in milliseconds')
axes[0,0].set_xlabel('Year');


axes[0,1].plot(lap1.year,lap1.fastest_lap)
axes[0,1].set_title('Monaco')
axes[0,1].set_ylabel('Fastest lap in milliseconds')
axes[0,1].set_xlabel('Year');

axes[0,2].plot(lap2.year,lap2.fastest_lap)
axes[0,2].set_title('Monza')
axes[0,2].set_ylabel('Fastest lap in milliseconds')
axes[0,2].set_xlabel('Year');

axes[1,0].plot(lap3.year,lap3.fastest_lap)
axes[1,0].set_title('Catalunya')
axes[1,0].set_ylabel('Fastest lap in milliseconds')
axes[1,0].set_xlabel('Year')

axes[1,1].plot(lap4.year,lap4.fastest_lap)
axes[1,1].set_title('Hungaroring')
axes[1,1].set_ylabel('Fastest lap in milliseconds')
axes[1,1].set_xlabel('Year');

axes[1,2].plot(lap5.year,lap5.fastest_lap)
axes[1,2].set_title('Spa')
axes[1,2].set_ylabel('Fastest lap in milliseconds')
axes[1,2].set_xlabel('Year');

plt.tight_layout(pad=2);


####  Some of the most common change in lap time seen in all the circuits over the years are that cars were quicker in 2004 and 2005 and there was a general increase in lap time till 2010 we need to dig deeper for the reasons of why this happened as it might be due number of different reasons from change in technical and safety rules to change in circuit layout.

#### Another common thing that is observed in every circuit is the peak of highest lap time was in 2014, this was when there was a major change in engine regulation starting the hybrid era.Formula 1 had a huge make over in 2014 in an effort to work towards much cleaner energy by taking the first step by replacing the traditional 2.4-liter V-8 engines by new 1.6-liter V-6 power units with energy recovery system.Though cars initially suffered due to increase in their lap time the teams quickly adapted to the new regulation and perfecting their cars to go fast as we can see after 2014 peak the lap time has decreased rapidly every year.

## **Hybrid Era**

#### Trying to find the car that has the best reliabilty in hybrid era

In [ ]:
# Extracting the data of hybrid era alone.

hybrid_era_results = merged_df[(merged_df['year'].between(2014,2021, inclusive = 'both'))]

In [ ]:
hybrid_era_results.positionText.unique()

In [ ]:
# Getting the number of races finished by the constructor by using the to numeric function as reliability issues and represented in text form.


races_finished = pd.to_numeric(hybrid_era_results['positionText'], errors = 'coerce').groupby(hybrid_era_results['constructors_name']).count()
races_finished = races_finished.reset_index()

In [ ]:
# Total number of race entries by the constructor 

races_entries = hybrid_era_results.groupby('constructors_name')[['raceId']].count().reset_index()

entries_finishes = races_finished.merge(races_entries, on = 'constructors_name')

entries_finishes = entries_finishes.rename(columns ={'positionText': 'races_finished', 'raceId': 'total_entries'})

# Calculating the reliability percentage 

entries_finishes['reliablility'] =  (entries_finishes['races_finished']/entries_finishes['total_entries'])*100



In [ ]:
# Plotting a graph  

plt.figure(figsize = (20,10))
plt.title('Reliability of Cars in Hybrid Era')
a =sns.barplot(x= 'reliablility', y  = 'constructors_name',data = entries_finishes);

#### Mercedes is the most reliable car closely followed by Alfa Romeo.

### Getting the distribution of points scored by drivers over the years in hybrid era.

In [ ]:
hybrid_era_results1 =  pd.pivot_table(hybrid_era_results , index = 'driver_name', columns = 'year' , values = 'points' , aggfunc= np.sum)

In [ ]:
plt.figure(figsize = (20,30))
plt.title("Points by drivers in hybrid era")
sns.heatmap(hybrid_era_results1, cmap = 'Blues')

## Inferences and Conclusion

### In this data analysis project we have explored some basic analysis with the F1 data from comparing the drivers and teams who has performed consistently in top level over the years to circuit analysis.
### Drivers like Lewis Hamilton, Michael Schumacher, Fernando Alonso,Sebastian Vettel and constructor teams Ferrari, Mercedes, Redbull have all featured top in almost all the analysis we performed here, implying these players are an integral part of Formula One.
### There is more room to do analysis with this formula one data from analysing the driver age, country,constructor's base country to lap time analysis which i hope to do in the future.